In [31]:
from ultralytics import YOLO
model = YOLO("modelFolder/model_v2.pt")
results = model(source="validimage/4_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))
    


image 1/1 c:\Team_Project-1\validimage\4_Color.png: 384x640 1 box, 368.0ms
Speed: 0.0ms preprocess, 368.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


1


In [32]:
from crop_ply import *
xyz_output= cropPly(image_points,"validimage/4.ply")
print(xyz_output)

[PointCloud with 1542 points.]


In [33]:
import open3d as o3d
import numpy as np

# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)

    # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.07)  # size를 조절하여 좌표 프레임 크기 설정 가능

    # 중앙 위치에 좌표 프레임 메시를 이동
    mesh.translate(center)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh])

    # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)
    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # 주요 주성분 벡터와 각각의 축과의 각도를 계산합니다.
    x_axis = np.array([1, 0, 0])  # x 축
    y_axis = np.array([0, 1, 0])  # y 축
    z_axis = np.array([0, 0, 1])  # z 축

    angle_x_rad = np.arccos(np.dot(eigenvectors[:, -1], x_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(x_axis)))
    angle_x_deg = np.degrees(angle_x_rad)

    angle_y_rad = np.arccos(np.dot(eigenvectors[:, -1], y_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(y_axis)))
    angle_y_deg = np.degrees(angle_y_rad)

    angle_z_rad = np.arccos(np.dot(eigenvectors[:, -1], z_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(z_axis)))
    angle_z_deg = np.degrees(angle_z_rad)

    
    print(f"요피치 롤(x 축 기준): {angle_x_deg}도")
    print(f"요피치 롤(y 축 기준): {angle_y_deg}도")
    print(f"요피치 롤(z 축 기준): {angle_z_deg}도")
    
    o3d.io.write_point_cloud("robot/test.ply", cloud)


[  -0.039082   -0.023089    -0.64577]
요피치 롤(x 축 기준): 142.55568152288373도
요피치 롤(y 축 기준): 127.43763680618306도
요피치 롤(z 축 기준): 90.60794392896534도


In [34]:
import open3d as o3d
import numpy as np
from scipy.spatial.transform import Rotation as R

# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)

    # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.07)  # size를 조절하여 좌표 프레임 크기 설정 가능

    # 중앙 위치에 좌표 프레임 메시를 이동
    mesh.translate(center)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh])

    # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)

    # 쿼터니언 회전을 계산하는 함수
    def compute_quaternion_rotation(points):
        covariance_matrix = np.cov(points, rowvar=False)
        eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
        longest_eigenvector = eigenvectors[:, -1]
        z_axis = np.array([0, 0, 1])  # z 축
        angle_z_rad = np.arccos(np.dot(longest_eigenvector, z_axis) / (np.linalg.norm(longest_eigenvector) * np.linalg.norm(z_axis)))
        rotation = R.from_euler('z', np.degrees(angle_z_rad), degrees=True)
        return rotation.as_quat()

    # 쿼터니언 회전을 계산합니다.
    quaternion_rotation = compute_quaternion_rotation(points)

    # 쿼터니언 회전을 출력합니다.
    print("쿼터니언 회전:", quaternion_rotation)

    o3d.io.write_point_cloud("robot/test.ply", cloud)


[  -0.039082   -0.023089    -0.64577]
쿼터니언 회전: [          0           0     0.71085     0.70335]


In [35]:
import open3d as o3d
import numpy as np

# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)

    # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.07)  # size를 조절하여 좌표 프레임 크기 설정 가능

    # 중앙 위치에 좌표 프레임 메시를 이동
    mesh.translate(center)

    # OBB 상자의 중심에서 시작하는 직교하는 선을 생성
    line_length = 0.1  # 선의 길이 조절
    obb_axes = -np.asarray(obb.R)  # OBB의 방향 벡터를 역방향으로 사용

    line_x = o3d.geometry.LineSet()
    line_x.points = o3d.utility.Vector3dVector([obb_center, obb_center + line_length * obb_axes[0]])
    line_x.lines = o3d.utility.Vector2iVector([[0, 1]])
    line_x.colors = o3d.utility.Vector3dVector([[1, 0, 0]])  # 빨간색

    line_y = o3d.geometry.LineSet()
    line_y.points = o3d.utility.Vector3dVector([obb_center, obb_center + line_length * obb_axes[1]])
    line_y.lines = o3d.utility.Vector2iVector([[0, 1]])
    line_y.colors = o3d.utility.Vector3dVector([[0, 1, 0]])  # 녹색

    line_z = o3d.geometry.LineSet()
    line_z.points = o3d.utility.Vector3dVector([obb_center, obb_center + line_length * obb_axes[2]])
    line_z.lines = o3d.utility.Vector2iVector([[0, 1]])
    line_z.colors = o3d.utility.Vector3dVector([[0, 0, 1]])  # 파란색

    

    # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)
    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # 주요 주성분 벡터와 각각의 축과의 각도를 계산합니다.
    x_axis = np.array([1, 0, 0])  # x 축
    y_axis = np.array([0, 1, 0])  # y 축
    z_axis = np.array([0, 0, 1])  # z 축

    angle_x_rad = np.arccos(np.dot(eigenvectors[:, -1], x_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(x_axis)))
    angle_x_deg = np.degrees(angle_x_rad)

    angle_y_rad = np.arccos(np.dot(eigenvectors[:, -1], y_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(y_axis)))
    angle_y_deg = np.degrees(angle_y_rad)

    angle_z_rad = np.arccos(np.dot(eigenvectors[:, -1], z_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(z_axis)))
    angle_z_deg = np.degrees(angle_z_rad)

    print(f"요피치 롤(x 축 기준): {angle_x_deg}도")
    print(f"요피치 롤(y 축 기준): {angle_y_deg}도")
    print(f"요피치 롤(z 축 기준): {angle_z_deg}도")
    
    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh, line_x, line_y, line_z])
    
    o3d.io.write_point_cloud("robot/test.ply", cloud)


[  -0.039082   -0.023089    -0.64577]
요피치 롤(x 축 기준): 142.55568152288373도
요피치 롤(y 축 기준): 127.43763680618306도
요피치 롤(z 축 기준): 90.60794392896534도
